In [1]:
import os
from google.cloud import client, storage 

In [2]:
# Import Google Credentials in Environ Varaibles
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"infared-81804c88c517.json"

In [3]:
bucket_name = "anvibucket"

In [4]:
 storage_client = storage.Client()

class Bucket:
    
    def __init__(self, bucket):
        self.bucket_name = bucket
        self.bucket = storage_client.get_bucket(bucket)
    
    def objects_list(self):
        object_files = [ i.name for i in list(self.bucket.list_blobs())]
        print(object_files)
    
    def upload(self, upload_file):
        filename = '{}'.format(str(upload_file.split("/")[-1]))
        create_blob = self.bucket.blob(filename)
        with open(upload_file, "rb") as f:
            create_blob.upload_from_file(f)
            
            print("-----   upload finished and make object to public and public_url  __________")
            create_blob.make_public()
            print("Object Public Url", create_blob.public_url)
            print("File Meta Details", create_blob.self_link)
            print("Bucket URL  gs://{0}/{1}".format(self.bucket_name, filename))
        
        return "gs://{0}/{1}".format(self.bucket_name, filename)

In [5]:
b = Bucket(bucket_name)
b.objects_list()

upload_file = './files/Invoice_607940909.pdf'
b.upload(upload_file)

['./files/Invoice_607940909.pdf', '/MY.PNG', 'Invoice_607940909.pdf', 'MY.PNG', 'SUNIL_RESUME.pdf', 'SUNIL_RESUME.pdfoutput-1-to-2.json', 'arun.PNG', 'download2.JPG', 'saini.docx']
-----   upload finished and make object to public and public_url  __________
Object Public Url https://storage.googleapis.com/anvibucket/Invoice_607940909.pdf
File Meta Details https://www.googleapis.com/storage/v1/b/anvibucket/o/Invoice_607940909.pdf
Bucket URL  gs://anvibucket/Invoice_607940909.pdf


'gs://anvibucket/Invoice_607940909.pdf'

In [6]:
#  Implement Google Vision API
from google.cloud import vision as VCF


class GoogleVision:
    
    def __init__(self, bucket, batch_size=2, client=None):
        self.bucket = storage_client.get_bucket(bucket)
        self.vision = VCF
        self.batch_size = batch_size
        self.client = VCF.ImageAnnotatorClient()
        
    def get_source_document_url(self, url):
        get_file = self.vision.GcsSource(uri=url)
        input_file_config = vision.InputConfig(gcs_source=get_file)
        return input_file_config
    
    def get_source_document_url(self, dest):
        gcs_destination = vision.GcsDestination(uri=dest)
        output_file_config = vision.OutputConfig(gcs_destination=gcs_destination, batch_size=self.batch_size)
        return output_file_config
    
    def get_result(self):
        async_request = self.vision.AsyncAnnotateFileRequest(
            features=[feature], input_config=input_config,output_config=output_config
        )
        operation = self.client.async_batch_annotate_files(requests=[async_request])
        operation.result(timeout=180)
        